In [1]:
%matplotlib inline
import time
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from scipy import io
import pickle

# Own code
sys.path.append("../")
from utils.data_utils import generate_dgp_tvp_var, generate_matrices, generate_contemp_matrices
from utils.tvp_models import TVPVARModel, tvp_ar_contemp, tvp_ar_non_contemp

# Suppress scientific notation in numpy
np.set_printoptions(suppress=True)

In [2]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    prior_train = train-10
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [3]:
%%time
import time
import os
from multiprocessing import Pool, Array

# They are going to be some disgusting warnings for the first iterations of the model (has to do with initialization)
import warnings
warnings.filterwarnings("ignore")

np.random.seed(12345)
n_iterations = 200

iterations_set = np.arange(1,n_iterations+1,1)
M = (3,7)
prior_list = ["svss"]#, "lasso_alternative", "horseshoe"]
args_tuples = [x for sub_list in [x for sub_list in [[[(n, m, prior) for n in iterations_set] for m in M] for prior in prior_list] for x in sub_list] for x in sub_list]

if __name__ == '__main__':
    
    pool = Pool()
    simulation_results = pool.starmap(simulation_run, args_tuples)

Run: 106, M: 3 & prior: svss -> MSD: 0.036393804151307704 | MSFE: 1.7768908655520845 | ALPL: -35.3222013382142
Run: 50, M: 3 & prior: svss -> MSD: 0.04472071294896369 | MSFE: 3.0840579099911647 | ALPL: -36.154268306916954
Run: 22, M: 3 & prior: svss -> MSD: 0.04818307247400092 | MSFE: 4.3735588697937775 | ALPL: -35.57950025277133
Run: 92, M: 3 & prior: svss -> MSD: 0.055645715735443226 | MSFE: 5.682737661574468 | ALPL: -34.87104573728996
Run: 64, M: 3 & prior: svss -> MSD: 0.03492710844717001 | MSFE: 3.7608509022612653 | ALPL: -35.79603825059975
Run: 78, M: 3 & prior: svss -> MSD: 0.01436480928311202 | MSFE: 2.342369883825156 | ALPL: -36.0445105894413
Run: 43, M: 3 & prior: svss -> MSD: 0.05987022479015047 | MSFE: 2.060894232007592 | ALPL: -34.60289532850386
Run: 85, M: 3 & prior: svss -> MSD: 0.06696280929843637 | MSFE: 7.109571237418656 | ALPL: -33.244684741105374
Run: 71, M: 3 & prior: svss -> MSD: 0.04308245446451637 | MSFE: 1.1562180062870124 | ALPL: -35.840268163795244
Run: 99, M

In [5]:
T = 200
train = 175
p = 1
sparsity = 0.40
prior = "svss"

simulation_parameters = [T,p,train, 1/3, 1/9, 4*1e-5, 1/6, 1/2, sparsity, 1e-2, 1e-9, 12345]

dump_to_disk = [simulation_results, simulation_parameters]

with open(f'../simulations/results/statistics_{T}_{p}_{n_iterations}_{prior}_{sparsity}.pkl', 'wb') as f:
        pickle.dump(dump_to_disk, f, pickle.HIGHEST_PROTOCOL)

In [6]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation|
    T = 200
    p = 1
    train = 175
    sparsity = 0.20
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [7]:
%%time
import time
import os
from multiprocessing import Pool, Array

# They are going to be some disgusting warnings for the first iterations of the model (has to do with initialization)
import warnings
warnings.filterwarnings("ignore")

np.random.seed(12345)
n_iterations = 200

iterations_set = np.arange(1,n_iterations+1,1)
M = (3,7)
prior_list = ["svss"]#, "lasso_alternative", "horseshoe"]
args_tuples = [x for sub_list in [x for sub_list in [[[(n, m, prior) for n in iterations_set] for m in M] for prior in prior_list] for x in sub_list] for x in sub_list]

if __name__ == '__main__':
    
    pool = Pool()
    simulation_results = pool.starmap(simulation_run, args_tuples)

Run: 106, M: 3 & prior: svss -> MSD: 0.036393804151307704 | MSFE: 1.7768908655520845 | ALPL: -35.3222013382142
Run: 78, M: 3 & prior: svss -> MSD: 0.0121760671185693 | MSFE: 2.3713885115111824 | ALPL: -35.8839843986317
Run: 8, M: 3 & prior: svss -> MSD: 0.03671308830395939 | MSFE: 1.181050347705105 | ALPL: -36.066231399568416
Run: 22, M: 3 & prior: svss -> MSD: 0.05024242594667705 | MSFE: 4.558433591346705 | ALPL: -35.43677718867096
Run: 92, M: 3 & prior: svss -> MSD: 0.055645715735443226 | MSFE: 5.682737661574468 | ALPL: -34.87104573728996
Run: 64, M: 3 & prior: svss -> MSD: 0.03492710844717001 | MSFE: 3.7608509022612653 | ALPL: -35.79603825059975
Run: 99, M: 3 & prior: svss -> MSD: 0.05821942117228549 | MSFE: 1.6647396115820368 | ALPL: -35.703684016611675
Run: 1, M: 3 & prior: svss -> MSD: 0.06284723968234861 | MSFE: 2.073093821872143 | ALPL: -35.08280911140103
Run: 71, M: 3 & prior: svss -> MSD: 0.04308245446451637 | MSFE: 1.1562180062870124 | ALPL: -35.840268163795244
Run: 85, M: 3

In [8]:
T = 200
train = 175
p = 1
sparsity = 0.20
prior = "svss"

simulation_parameters = [T,p,train, 1/3, 1/9, 4*1e-5, 1/6, 1/2, sparsity, 1e-2, 1e-9, 12345]

dump_to_disk = [simulation_results, simulation_parameters]

with open(f'../simulations/results/statistics_{T}_{p}_{n_iterations}_{prior}_{sparsity}.pkl', 'wb') as f:
        pickle.dump(dump_to_disk, f, pickle.HIGHEST_PROTOCOL)

In [9]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 100
    p = 1
    train = 75
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [10]:
%%time
import time
import os
from multiprocessing import Pool, Array

# They are going to be some disgusting warnings for the first iterations of the model (has to do with initialization)
import warnings
warnings.filterwarnings("ignore")

np.random.seed(12345)
n_iterations = 200

iterations_set = np.arange(1,n_iterations+1,1)
M = (3,7)
prior_list = ["svss"]#, "lasso_alternative", "horseshoe"]
args_tuples = [x for sub_list in [x for sub_list in [[[(n, m, prior) for n in iterations_set] for m in M] for prior in prior_list] for x in sub_list] for x in sub_list]

if __name__ == '__main__':
    
    pool = Pool()
    simulation_results = pool.starmap(simulation_run, args_tuples)

Run: 50, M: 3 & prior: svss -> MSD: 0.038977399059032816 | MSFE: 1.242848634017649 | ALPL: -36.25452601866279
Run: 71, M: 3 & prior: svss -> MSD: 0.0780867104307407 | MSFE: 1.967169910053968 | ALPL: -35.82313452981859
Run: 36, M: 3 & prior: svss -> MSD: 0.03531146374425496 | MSFE: 1.2859027839974453 | ALPL: -36.21726260163655
Run: 78, M: 3 & prior: svss -> MSD: 0.039767144349225174 | MSFE: 1.4905935988376293 | ALPL: -35.42404901494804
Run: 85, M: 3 & prior: svss -> MSD: 0.04332812584499399 | MSFE: 1.3660186813545776 | ALPL: -35.17798282264047
Run: 43, M: 3 & prior: svss -> MSD: 0.042146653846474647 | MSFE: 1.8350041574635625 | ALPL: -36.39628678279439
Run: 8, M: 3 & prior: svss -> MSD: 0.0355693866288384 | MSFE: 3.2567132200284536 | ALPL: -34.930964044814885
Run: 92, M: 3 & prior: svss -> MSD: 0.06106246385746947 | MSFE: 1.2624686869652086 | ALPL: -35.62993178125713
Run: 57, M: 3 & prior: svss -> MSD: 0.04235782137968111 | MSFE: 1.6908143530088018 | ALPL: -35.837527380499665
Run: 22, M

In [11]:
T = 100
train = 75
p = 1
sparsity = 0.40
prior = "svss"

simulation_parameters = [T,p,train, 1/3, 1/9, 4*1e-5, 1/6, 1/2, sparsity, 1e-2, 1e-9, 12345]

dump_to_disk = [simulation_results, simulation_parameters]

with open(f'../simulations/results/statistics_{T}_{p}_{n_iterations}_{prior}_{sparsity}.pkl', 'wb') as f:
        pickle.dump(dump_to_disk, f, pickle.HIGHEST_PROTOCOL)

T = 100, SPARSITY = 0.20

In [12]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 100
    p = 1
    train = 75
    sparsity = 0.20
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [13]:
%%time
import time
import os
from multiprocessing import Pool, Array

# They are going to be some disgusting warnings for the first iterations of the model (has to do with initialization)
import warnings
warnings.filterwarnings("ignore")

np.random.seed(12345)
n_iterations = 200

iterations_set = np.arange(1,n_iterations+1,1)
M = (3,7)
prior_list = ["svss"]#, "lasso_alternative", "horseshoe"]
args_tuples = [x for sub_list in [x for sub_list in [[[(n, m, prior) for n in iterations_set] for m in M] for prior in prior_list] for x in sub_list] for x in sub_list]

if __name__ == '__main__':
    
    pool = Pool()
    simulation_results = pool.starmap(simulation_run, args_tuples)

Run: 50, M: 3 & prior: svss -> MSD: 0.036006789888990684 | MSFE: 1.1445346647353265 | ALPL: -35.88298513372895
Run: 57, M: 3 & prior: svss -> MSD: 0.04419617860507608 | MSFE: 1.7965744181764571 | ALPL: -35.80186828108815
Run: 71, M: 3 & prior: svss -> MSD: 0.0780867104307407 | MSFE: 1.967169910053968 | ALPL: -35.82313452981859
Run: 8, M: 3 & prior: svss -> MSD: 0.03489914210870156 | MSFE: 2.955145426688217 | ALPL: -35.09230897964278
Run: 36, M: 3 & prior: svss -> MSD: 0.03531146374425496 | MSFE: 1.2859027839974453 | ALPL: -36.21726260163655
Run: 78, M: 3 & prior: svss -> MSD: 0.039767144349225174 | MSFE: 1.4905935988376293 | ALPL: -35.42404901494804
Run: 43, M: 3 & prior: svss -> MSD: 0.042146653846474647 | MSFE: 1.8350041574635625 | ALPL: -36.39628678279439
Run: 22, M: 3 & prior: svss -> MSD: 0.043935795896542305 | MSFE: 6.27443751523951 | ALPL: -35.43086182264256
Run: 85, M: 3 & prior: svss -> MSD: 0.03886156651982403 | MSFE: 1.2147222122548516 | ALPL: -35.13192868671257
Run: 1, M: 3

In [14]:
T = 100
train = 75
p = 1
sparsity = 0.20
prior = "svss"

simulation_parameters = [T,p,train, 1/3, 1/9, 4*1e-5, 1/6, 1/2, sparsity, 1e-2, 1e-9, 12345]

dump_to_disk = [simulation_results, simulation_parameters]

with open(f'../simulations/results/statistics_{T}_{p}_{n_iterations}_{prior}_{sparsity}.pkl', 'wb') as f:
        pickle.dump(dump_to_disk, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
optimal_set = []

for run in range(1,11):

    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    prior_train = train-10
    sparsity = 0.05
    M = 2
    prior = "lasso_alternative"

    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")

    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)

    prior_parameters = None

    a0_set = np.linspace(1,5,5)
    b0_set = np.hstack(([1], np.linspace(3,12,4)))
    tpl_list = [x for sub_list in [[(a,b) for a in a0_set] for b in b0_set] for x in sub_list]

    msfe_list = []

    for parameters in tpl_list:

        optim_pior_parameters = {'a0_lasso':parameters[0],'b0_lasso':parameters[1]}
        msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(train, M, p, prior_train, X_matrix_contemp[:train], y_matrix_contemp[:,:train], prior, print_status=False, iterations=100, prior_parameters=optim_pior_parameters)

        msfe_list.append(msfe_contemp.mean())

    optimal_prior = tpl_list[msfe_list.index(min(msfe_list))]
    prior_parameters = {'a0_lasso':optimal_prior[0],'b0_lasso':optimal_prior[1]}

    print(f"Run: {run} -> Optimal prior: {optimal_prior}")
    optimal_set.append(optimal_prior)
    

In [ ]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    prior_train = train-10
    sparsity = 0.05
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
#     prior_parameters = None
    
#     if prior == 'lasso_alternative':
        
#         a0_set = np.linspace(1,5,5)
#         b0_set = np.hstack(([1], np.linspace(3,12,4)))
#         tpl_list = [x for sub_list in [[(a,b) for a in a0_set] for b in b0_set] for x in sub_list]
        
#         msfe_list = []
        
#         for parameters in tpl_list:
            
#             optim_pior_parameters = {'a0_lasso':parameters[0],'b0_lasso':parameters[1]}
#             msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(train, M, p, prior_train, X_matrix_contemp[:train], y_matrix_contemp[:,:train], prior, print_status=False, iterations=100, prior_parameters=optim_pior_parameters)
            
#             msfe_list.append(msfe_contemp.mean())
        
#         optimal_prior = tpl_list[msfe_list.index(min(msfe_list))]
#         prior_parameters = {'a0_lasso':optimal_prior[0],'b0_lasso':optimal_prior[1]}
        
#     elif prior == 'svss':
        
#         g0_set = np.linspace(1,5,5)
#         h0_set = np.hstack(([1], np.linspace(3,12,4)))
#         tpl_list = [x for sub_list in [[(g,h) for g in g0_set] for h in h0_set] for x in sub_list]
        
#         msfe_list = []
        
#         for parameters in tpl_list:
            
#             optim_pior_parameters = {'g0':parameters[0],'h0':parameters[1], 'pi0': 0.5}
#             msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(train, M, p, prior_train, X_matrix_contemp[:train], y_matrix_contemp[:,:train], prior, print_status=False, iterations=100, prior_parameters=optim_pior_parameters)
            
#             msfe_list.append(msfe_contemp.mean())
        
#         optimal_prior = tpl_list[msfe_list.index(min(msfe_list))]
#         prior_parameters = {'g0': optimal_prior[0], 'h0': optimal_prior[1], 'pi0': 0.5}
        
#     elif prior == 'horseshoe':
#         prior_parameters = {'a0': 1, 'b0': 1}
        
#         a0_set = np.hstack(([1.5], np.linspace(3,12,4)))
#         b0_set = np.hstack(([1.5],np.linspace(1,5,5)))
#         tpl_list = [x for sub_list in [[(a,b) for a in a0_set] for b in b0_set] for x in sub_list]
        
#         msfe_list = []
        
#         for parameters in tpl_list:
            
#             optim_pior_parameters = {'a0':parameters[0],'b0':parameters[1]}
#             msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(train, M, p, prior_train, X_matrix_contemp[:train], y_matrix_contemp[:,:train], prior, print_status=False, iterations=100, prior_parameters=optim_pior_parameters)
            
#             msfe_list.append(msfe_contemp.mean())
        
#         optimal_prior = tpl_list[msfe_list.index(min(msfe_list))]
#         prior_parameters = {'a0':optimal_prior[0],'b0':optimal_prior[1]}
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]